# Programando con Python Spark (PySpark)

Recordando:
- El driver program accesa al ambiente de Spark mediante un objeto SparkContext
- El tipo de dato y concepto clave en Spark es el dataset llamados RDD
- Cargamos datos en un RDD y hacemos operaciones

## ¡Nuestro primer programa!

In [2]:
# !wget https://github.com/apache-spark/spark/blob/master/README.md

In [4]:
from pyspark import SparkContext

In [5]:
sc = SparkContext(master = "local[*]")

In [6]:
lineas = sc.textFile("README.md", 4)

In [7]:
lineas.count()

1140

In [8]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [9]:
lineasPython.first()

'high-level APIs in Scala, Java, and Python, and an optimized engine that'

----

## RDD

**R**esilient **D**istributed **D**ateset (**RDD**)

Recordando y agregando:
- Contiene **Datos distribuidos** mediante **particiones** (de Workers)
- Habilita **operaciones** para su **ejecución en paralelo**
- Son **inmutables**
- **En caso de pérdida, la computación ejecutada se re-ejecuta**


Tres maneras de crear RDDs:

- Mediante un dataset externo:

In [10]:
lineas = sc.textFile("README.md", 4)

- Distribuyendo una colección:

In [11]:
lineas = sc.parallelize([1, 2, 3])

- Transformando un RDD existente:

In [12]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

### Operaciones sobre RDDs

#### Transformaciones

Crean un nuevo RDD a partir de otro previo.

P. ej.:
*map()*


#### Acciones

Corre/ejecuta/computa un resultado basado en un RDD existente.

P. ej.:
*count()*

## Programación Funcional con Python

- Muchas transformaciones y algunas acciones esperan una función
- En algunos casos, pueden ser funciones para operaciones más complejas
- Para funciones simples, una expresión lambda es conveniente:
```python
>>> lambda line: “Python” in line
```


### map()

- Lee un elemento a la vez
- Toma un valor, crea un nuevo valor


In [13]:
rdd = sc.parallelize([1, 2, 3, 4])

In [29]:
rdd.map(lambda x: x * 2)

PythonRDD[23] at RDD at PythonRDD.scala:48

In [30]:
rdd.map(lambda x: x * 2).collect()

[2, 4, 6, 8]

### filter()

- Lee un elemento a la vez
- Evalua cada elemento
- Regresa los elementos que pasan el filtro  (filtro)

In [31]:
rdd = sc.parallelize([1, 2, 3, 4])

In [32]:
rdd.filter(lambda x: x % 2 == 0)

PythonRDD[26] at RDD at PythonRDD.scala:48

In [33]:
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

### flatMap()

Produce multiples elementos por cada elemento de entrada

In [34]:
rdd = sc.parallelize([1,2,3])

In [37]:
rdd.map(lambda x: [x, x * 2])

PythonRDD[31] at RDD at PythonRDD.scala:48

In [40]:
rdd.flatMap(lambda x: [x, x * 2])

PythonRDD[34] at RDD at PythonRDD.scala:48

# Transformations are lazy!
## Featuring: Lazy evaluation!! 🔥🙈

[Haskell Lazy Evaluation](https://wiki.haskell.org/Lazy_evaluation):

>Lazy evaluation is a method to evaluate a Haskell program. It means that expressions are not evaluated when they are bound to variables, but their evaluation is deferred until their results are needed by other computations. In consequence, arguments are not evaluated before they are passed to a function, but only when their values are actually used. 

[The Incomplete Guide to Lazy Evaluation (in Haskell)](https://hackhands.com/guide-lazy-evaluation-haskell/):

> Originally, I wanted to write a complete guide to lazy evaluation, but then.

>Lazy evaluation is the most widely used method for executing Haskell program code on a computer. It determines the time and memory usage of Haskell programs, and it allows new and powerful ways to write modular code. To make full use of purely functional programming, a good understanding of lazy evaluation is very helpful.


Un RDD solo es ejecutado cuando las acciones corren sobre el mismo:

In [46]:
lineas = sc.textFile("README.md", 4)
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [47]:
lineasPython

PythonRDD[39] at RDD at PythonRDD.scala:48

In [48]:
lineasPython.first()

'high-level APIs in Scala, Java, and Python, and an optimized engine that'

Al usar la evaluación floja, Spark puede contener en memoria RDD que se procesa unicamente cuando se le requiere. Sin la necesidad de cargar a memoria todas las lineas conteniendo "Python".


💻🐍

## Acciones

- Las acciones causan transformaciones para ser almacenadas en RDDs nuevos
- También regresan resultados a ambas partes: el *driver* o un almacenamiento externo
- Los RDDs son re-calculados por cada acción que se les ejecuta
- Pueden ser almacenados para un uso posterior: `rdd.persist()`

### count()

Obtiene las instancias en el RDD:

In [49]:
rdd = sc.parallelize([1, 2, 3, 4])

In [50]:
rdd

ParallelCollectionRDD[42] at parallelize at PythonRDD.scala:480

In [51]:
rdd.count()

4

### collect()
- `collect()` recupera el RDD completo 🚒
- Útil para inspeccionar datasets pequeños de manera local y para unit-testing
- **LOS RESULTADOS DEBEN CABER EN LA MEMORIA DEL EQUIPO LOCAL**

In [52]:
rdd = sc.parallelize([1, 2, 3])

In [53]:
rdd

ParallelCollectionRDD[44] at parallelize at PythonRDD.scala:480

In [54]:
rdd.collect()

[1, 2, 3]

### take(), takeSample(), first(), top(), takeOrdered() 

- ```take(n)``` regresa los primeros *n* elementos de un RDD
- ```take(n)``` puede obtener resultados sezgados. Su uso es adecuado solo para pruebas o debugging
- ```takeSample()``` como el nombre lo indica, es el más adecuado para tomar una muesta del dataset
- ```first(n)``` al igual que ```take(n)```, obtiene los primeros *n* elementos de un RDD
- ```top(), takeOrdered()``` como métodos más formales para obtener elementos ordenados de un RDD

### takeOrdered()

In [59]:
rdd = sc.parallelize([5, 1, 3, 2])

In [60]:
rdd.takeOrdered(4)

[1, 2, 3, 5]

In [61]:
rdd.takeOrdered(4, lambda n: -n)

[5, 3, 2, 1]

### reduce()
Toma dos elementos del mismo tipo y regresa un nuevo elemento:

In [62]:
rdd = sc.parallelize([1,2,3])

In [63]:
rdd.reduce(lambda x, y:  x*y)

6

## Persistencia
- Spark re-calcula los RDDs cada vez que se llama a una acción:
    - Esto puede ser caro y también causar un tráfico innecesario desde el disco (lectura)
- Podemos evitar esto almacenando datos en caché con ```persist()```.

In [64]:
lineas = sc.textFile("README.md", 4)

In [65]:
lineas.count()

1140

In [66]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [69]:
# Causa a Spark el recargar la variable "lineas" desde el disco 🙊
lineasPython.count()

3

### Aplicando persistencia

In [70]:
lineas = sc.textFile("README.md", 4)

In [71]:
lineas.persist() # Ahora, este RDD se mantiene en RAM

README.md MapPartitionsRDD[57] at textFile at NativeMethodAccessorImpl.java:0

In [72]:
lineas.count()

1140

In [73]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [75]:
# Spark no volverá a hacer el cómputo para "lineas" cada vez que es usado
lineasPython.count()

3

## Construyendo un Pipeline de operaciones para RDDs



```python
>>> lineas = sc.textFile("README.md")
>>> lineas.map(...).filter(...).count(...)



>>> lineas = sc.textFile("README.md")
>>> (lineas
     .map(...)
     .filter(...)
     .count(...))
```

In [76]:
lineas = sc.textFile("README.md")

In [82]:
lineas.filter(lambda line :  "Python" in line).count()

3